# Clonamos el repositorio con los modelos y herramientas

In [1]:
!git clone https://github.com/dannasalazar11/Msc_thesis.git

Cloning into 'Msc_thesis'...
remote: Enumerating objects: 298, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (100/100), done.
remote: Total 298 (delta 81), reused 52 (delta 26), pack-reused 172 (from 1)
Receiving objects: 100% (298/298), 49.92 MiB | 41.03 MiB/s, done.
Resolving deltas: 100% (172/172), done.


In [2]:
import sys
sys.path.append('/kaggle/working/Msc_thesis')

from gmrrnet_adhd.utils import get_segmented_data, train_L24O_cv
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy('mixed_float16')

import tensorflow as tf
import numpy as np
import random
import os

# Establecer semilla
seed = 42

# Semillas para módulos principales
np.random.seed(seed)
random.seed(seed)
tf.random.set_seed(seed)

2025-04-30 20:13:24.062034: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746044004.263741      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746044004.320251      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


# Importar base de datos segmentada (Segmentos de 4 seg con translape del 50%, es decir, de 2 seg)

In [3]:
X, y, sbjs = get_segmented_data()
X.shape, y.shape, len(sbjs)

((8213, 19, 512), (8213, 2), 8213)

# Importamos el modelo y definimos los hiperparámetros

In [4]:
from tensorflow.keras.losses import CategoricalCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from gmrrnet_adhd.models.cnn_lstm_eegnet import cnn_lstm_eegnet

model_name = 'CNN_LSTM_EEGNet'
model_args = {
    "n_channels": 19,      # EEG electrodes
    "n_times": 512,        # time points per trial
    "n_classes": 2,        # binary task
    "temporal_kernel": 25, # ≈200 ms at 128 Hz
    "pool_size": 20,
    "pool_stride": 10
}

compile_args = {
    'loss': CategoricalCrossentropy(),
    'optimizer': lambda: Adam(1e-2),  # función que retorna un nuevo optimizador
    'metrics': ['categorical_accuracy']
}


model = cnn_lstm_eegnet(**model_args)

model.summary()

I0000 00:00:1746044020.497139      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1746044020.497834      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Model: "cnn_lstm_eegnet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 19, 512, 1)     │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ cast (Cast)               │ (None, 19, 512, 1)     │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 19, 512, 50)    │          1,250 │ cast[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 19, 512, 50)    │            200 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 1, 512, 100)    │              0 │ batch_normalization[0… │
│                           │                        │                │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ depthwise_conv2d          │ (None, 1, 512, 100)    │          1,900 │ activation[0][0]       │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 1, 512, 100)    │            400 │ depthwise_conv2d[0][0] │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ average_pooling2d         │ (None, 1, 50, 100)     │              0 │ activation[1][0]       │
│ (AveragePooling2D)        │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dropout (Dropout)         │ (None, 1, 50, 100)     │              0 │ average_pooling2d[0][… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ permute (Permute)         │ (None, 50, 1, 100)     │              0 │ dropout[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ time_distributed          │ (None, 50, 100)        │              0 │ permute[0][0]          │
│ (TimeDistributed)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm (LSTM)               │ (None, 50, 10)         │          4,440 │ time_distributed[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ lstm_1 (LSTM)             │ (None, 10)             │            840 │ lstm[0][0]             │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 2)              │             22 │ lstm_1[0][0]           │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 9,052 (35.36 KB)

 Trainable params: 8,752 (34.19 KB)

 Non-trainable params: 300 (1.17 KB)

# Resultados - Leave 24 Subjects Out

In [5]:
import os

import pickle

with open("/kaggle/input/ieee-tdah-control-database/folds.pkl", "rb") as f:
    folds = pickle.load(f)

In [6]:
results = train_L24O_cv(cnn_lstm_eegnet, X, y, sbjs, model_args, compile_args, folds)

Fold 1/5. Test subjects: ['v28p', 'v274', 'v1p', 'v231', 'v22p', 'v29p', 'v206', 'v238', 'v31p', 'v35p', 'v177', 'v200', 'v112', 'v113', 'v48p', 'v140', 'v131', 'v125', 'v55p', 'v143', 'v43p', 'v305', 'v134', 'v114']


I0000 00:00:1746044030.524256      70 cuda_dnn.cc:529] Loaded cuDNN version 90300


46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 23ms/step
Fold metrics: {'accuracy': 0.7886067261496225, 'recall': 0.7905168637757337, 'precision': 0.790976135547067, 'kappa': 0.5783367097266436, 'auc': 0.7905168637757337}
Fold 2/5. Test subjects: ['v18p', 'v39p', 'v234', 'v32p', 'v190', 'v6p', 'v254', 'v204', 'v24p', 'v183', 'v246', 'v219', 'v298', 'v41p', 'v47p', 'v308', 'v52p', 'v300', 'v59p', 'v299', 'v302', 'v51p', 'v109', 'v127']
53/53 ━━━━━━━━━━━━━━━━━━━━ 2s 19ms/step
Fold metrics: {'accuracy': 0.7254196642685852, 'recall': 0.7299951759608364, 'precision': 0.727027901857966, 'kappa': 0.45244290008644006, 'auc': 0.7299951759608363}
Fold 3/5. Test subjects: ['v215', 'v3p', 'v209', 'v37p', 'v213', 'v15p', 'v284', 'v181', 'v19p', 'v34p', 'v263', 'v244', 'v138', 'v121', 'v46p', 'v54p', 'v120', 'v310', 'v147', 'v50p', 'v56p', 'v107', 'v297', 'v108']
56/56 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step
Fold metrics: {'accuracy': 0.9015837104072398, 'recall': 0.8789842744006993, 'precision': 0.9235467895079835, 'kapp

In [7]:
import numpy as np

accs = []
for result in results:
    accs.append(result['accuracy'])

np.mean(accs)

0.8579151048503615

In [8]:
import pickle

with open(f'results_L24SO_{model_name}.pkl', 'wb') as f:
    pickle.dump(results, f)